In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
import keras 

import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# -- Keras Import
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.preprocessing import image

from keras.datasets import imdb
from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

from keras.utils import np_utils
from keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding
from keras.layers import LSTM, GRU, SimpleRNN

from keras.layers import Activation, TimeDistributed, RepeatVector
from keras.callbacks import EarlyStopping, ModelCheckpoint

2023-03-22 19:00:07.985693: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df1 = pd.read_csv('/Users/vipin-16319/Downloads/train-sample.csv')
df1.head()
df = df1.sample(n=10000)

In [3]:
df = df.drop(['PostId','PostCreationDate','OwnerUserId','OwnerCreationDate','ReputationAtPostCreation','OwnerUndeletedAnswerCountAtPostTime','PostClosedDate'],axis=1)

In [4]:
df

,Title,BodyMarkdown,Tag1,Tag2,Tag3,Tag4,Tag5,OpenStatus
23330,Android Widget Bitmap Sizes,"Hallo,\r\n\r\nI am writing a widget, but am un...",android,bitmap,android-widget,android-layout,android-manifest,open
45699,StackOverflow's 404 Error,I find the StackOverflow's `404` error unexpec...,http-status-code-404,NaN,NaN,NaN,NaN,off topic
128700,exact cost of the EC2 instance for 1 hour in A...,I want to know exact cost of an windows and li...,ami,NaN,NaN,NaN,NaN,off topic
118080,jquery focus on input,I got one main search field always visible at ...,jquery,focus,html,NaN,NaN,open
74160,get the sub index in foreach loop in simpleXML...,"<?xml version=""1.0"" encoding=""utf-8""?>\r\n...",php,foreach,key,NaN,NaN,open
...,...,...,...,...,...,...,...,...
23908,Different return values from calling stored pr...,<?php\r\n error_reporting(E_ALL);\r\n\t...,php,mysql,stored-procedures,NaN,NaN,open
84748,records migration error in SAP ByDesign,i am trying to migrate supplier records from o...,sap,NaN,NaN,NaN,NaN,not a real question
11692,Spoon Studio crashes or please suggest another...,"I have downloaded trial of Spoon Studio, yet i...",c#,.net,obfuscation,NaN,NaN,not constructive
116280,Bing map silverlight binding for moving target,"I want to animate a ""car"" dot on a Bing map. I...",silverlight,binding,bing,NaN,NaN,open


In [5]:
df['OpenStatus']= df['OpenStatus'].map({'open': 0, 'not a real question': 1, 'off topic': 2, 'not constructive': 3, 'too localized': 4}) 
df.head()

,Title,BodyMarkdown,Tag1,Tag2,Tag3,Tag4,Tag5,OpenStatus
23330,Android Widget Bitmap Sizes,"Hallo,\r\n\r\nI am writing a widget, but am un...",android,bitmap,android-widget,android-layout,android-manifest,0
45699,StackOverflow's 404 Error,I find the StackOverflow's `404` error unexpec...,http-status-code-404,NaN,NaN,NaN,NaN,2
128700,exact cost of the EC2 instance for 1 hour in A...,I want to know exact cost of an windows and li...,ami,NaN,NaN,NaN,NaN,2
118080,jquery focus on input,I got one main search field always visible at ...,jquery,focus,html,NaN,NaN,0
74160,get the sub index in foreach loop in simpleXML...,"<?xml version=""1.0"" encoding=""utf-8""?>\r\n...",php,foreach,key,NaN,NaN,0


In [6]:
df['Tag1']=df['Tag1'].replace(np.nan,' ')
df['Tag2']=df['Tag2'].replace(np.nan,' ')
df['Tag3']=df['Tag3'].replace(np.nan,' ')
df['Tag4']=df['Tag4'].replace(np.nan,' ')
df['Tag5']=df['Tag5'].replace(np.nan,' ')
df['Tags']=df['Tag1']+' '+df['Tag2']+' '+df['Tag3']+' '+df['Tag4']+' '+df['Tag5']

In [7]:
df=df.drop(['Tag1','Tag2','Tag3','Tag4','Tag5'],axis=1)
df.head()

,Title,BodyMarkdown,OpenStatus,Tags
23330,Android Widget Bitmap Sizes,"Hallo,\r\n\r\nI am writing a widget, but am un...",0,android bitmap android-widget android-layout a...
45699,StackOverflow's 404 Error,I find the StackOverflow's `404` error unexpec...,2,http-status-code-404
128700,exact cost of the EC2 instance for 1 hour in A...,I want to know exact cost of an windows and li...,2,ami
118080,jquery focus on input,I got one main search field always visible at ...,0,jquery focus html
74160,get the sub index in foreach loop in simpleXML...,"<?xml version=""1.0"" encoding=""utf-8""?>\r\n...",0,php foreach key


In [8]:
def emoji(tweet):
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\)|:O)', ' positiveemoji ', tweet)
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' positiveemoji ', tweet)
    tweet = re.sub(r'(<3|:\*)', ' positiveemoji ', tweet)
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;|@-\))', ' positiveemoji ', tweet)
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:|:-/|:-\|)', ' negetiveemoji ', tweet)
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' negetiveemoji ', tweet)
    return tweet

In [9]:
text1=df.columns[0]
text2=df.columns[1]
text3=df.columns[3]

In [10]:
import re

def process_tweet(tweet):
    tweet = tweet.lower()                                             
    tweet = re.sub('@[^\s]+', '', tweet)                              
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', tweet)  
    tweet = re.sub(r"\d+", " ", str(tweet))                          
    tweet = re.sub('&quot;'," ", tweet)                              
    tweet = emoji(tweet)                                             
    tweet = re.sub(r"\b[a-zA-Z]\b", "", str(tweet))                   
    tweet = re.sub(r"[^\w\s]", " ", str(tweet))                      
    tweet = re.sub(r'(..)\1+', r'\1', tweet)
    tweet = re.sub(r'\b[a-zA-Z]{2}\b', '', tweet)                      
    tweet = re.sub(r"\s+", " ", str(tweet))                             
    return tweet

In [11]:
!pip install contractions
df['Title'] = np.vectorize(process_tweet)(df[text1])
df['BodyMarkdown']=np.vectorize(process_tweet)(df[text2])
df['Tags']=np.vectorize(process_tweet)(df[text3])

In [12]:
import contractions 
df['Title'] = df['Title'].apply(lambda x: contractions.fix(x))
df['BodyMarkdown']=df['BodyMarkdown'].apply(lambda x: contractions.fix(x))
df['Tags']=df['Tags'].apply(lambda x: contractions.fix(x))

In [13]:
tokenized_text1 = df['Title'].apply(lambda x: x.split())
tokenized_text2=df['BodyMarkdown'].apply(lambda x: x.split())
tokenized_text3=df['Tags'].apply(lambda x: x.split())

In [14]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_text1 = tokenized_text1.apply(lambda x: [stemmer.stem(i) for i in x])
tokenized_text2 = tokenized_text2.apply(lambda x: [stemmer.stem(i) for i in x])
tokenized_text3 = tokenized_text3.apply(lambda x: [stemmer.stem(i) for i in x])

In [15]:
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

tokenized_text1 = tokenized_text1.apply(lambda x: [lemmatizer.lemmatize(i) for i in x])
tokenized_text2 = tokenized_text2.apply(lambda x: [lemmatizer.lemmatize(i) for i in x])
tokenized_text3 = tokenized_text3.apply(lambda x: [lemmatizer.lemmatize(i) for i in x])

In [16]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words, nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vipin-16319/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


({'a',
  'about',
  'above',
  'after',
  'again',
  'against',
  'ain',
  'all',
  'am',
  'an',
  'and',
  'any',
  'are',
  'aren',
  "aren't",
  'as',
  'at',
  'be',
  'because',
  'been',
  'before',
  'being',
  'below',
  'between',
  'both',
  'but',
  'by',
  'can',
  'couldn',
  "couldn't",
  'd',
  'did',
  'didn',
  "didn't",
  'do',
  'does',
  'doesn',
  "doesn't",
  'doing',
  'don',
  "don't",
  'down',
  'during',
  'each',
  'few',
  'for',
  'from',
  'further',
  'had',
  'hadn',
  "hadn't",
  'has',
  'hasn',
  "hasn't",
  'have',
  'haven',
  "haven't",
  'having',
  'he',
  'her',
  'here',
  'hers',
  'herself',
  'him',
  'himself',
  'his',
  'how',
  'i',
  'if',
  'in',
  'into',
  'is',
  'isn',
  "isn't",
  'it',
  "it's",
  'its',
  'itself',
  'just',
  'll',
  'm',
  'ma',
  'me',
  'mightn',
  "mightn't",
  'more',
  'most',
  'mustn',
  "mustn't",
  'my',
  'myself',
  'needn',
  "needn't",
  'no',
  'nor',
  'not',
  'now',
  'o',
  'of',
  'off',
 

In [18]:
tokenized_text1

23330                       [android, widget, bitmap, size]
45699                                [stackoverflow, error]
128700    [exact, cost, the, instanc, for, hour, amazon,...
118080                                [jqueri, focu, input]
74160      [get, the, sub, index, foreach, loop, simplexml]
                                ...                        
23908     [differ, return, valu, from, call, store, proc...
84748                [record, migrat, error, sap, bydesign]
11692     [spoon, studio, crash, plea, suggest, anoth, a...
116280    [bing, map, silverlight, bind, for, move, target]
60203     [enjoy, bug, fix, more, than, code, from, scra...
Name: Title, Length: 10000, dtype: object

In [19]:
for i, text in enumerate(tokenized_text1):
    tokenized_text1[i] = ' '.join([word for word in text if word not in stop_words])  
    
for i, text in enumerate(tokenized_text2):
    tokenized_text2[i] = ' '.join([word for word in text if word not in stop_words])
    
for i, text in enumerate(tokenized_text3):
    tokenized_text3[i] = ' '.join([word for word in text if word not in stop_words])

df['Title'] = tokenized_text1
df['BodyMarkdown'] = tokenized_text2
df['Tags'] = tokenized_text3
df.head()

,Title,BodyMarkdown,OpenStatus,Tags
23330,"[android, widget, bitmap, size]","[hallo, write, widget, but, unsur, what, size,...",0,"[android, bitmap, android, widget, android, la..."
45699,"[stackoverflow, error]","[find, the, stackoverflow, error, unexpectedli...",2,"[http, statu, code]"
128700,"[exact, cost, the, instanc, for, hour, amazon,...","[want, know, exact, cost, window, and, linux, ...",2,[ami]
118080,"[jqueri, focu, input]","[got, one, main, search, field, alway, visibl,...",0,"[jqueri, focu, html]"
74160,"[get, the, sub, index, foreach, loop, simplexml]","[xml, version, encod, utf, item, item, titl, t...",0,"[php, foreach, key]"


In [31]:
df
type(df['Combined_input'])

pandas.core.series.Series

In [32]:
import gensim.downloader as api
word2vec_model = api.load("word2vec-google-news-300")

In [33]:
import gensim

def word2vec_transform(text, word2vec_model, max_length):

    words = text.split()

    vectors = []
    for word in words:
        if word in word2vec_model:
            vectors.append(word2vec_model[word])
        else:
            vectors.append(np.zeros(word2vec_model.vector_size))

    if len(vectors) > max_length:
        vectors = vectors[:max_length]
    else:
        vectors += [np.zeros(word2vec_model.vector_size)] * (max_length - len(vectors))

    return np.array(vectors)

In [34]:
import numpy as np

max_length = 200


X = np.array([word2vec_transform(text, word2vec_model, max_length) for text in df['Combined_input']])


In [37]:
X[0]

array([[-0.08691406, -0.12792969, -0.33789062, ...,  0.03125   ,
        -0.22070312,  0.07470703],
       [ 0.19628906, -0.31054688, -0.14941406, ..., -0.23339844,
        -0.28125   ,  0.06933594],
       [ 0.16113281,  0.10986328,  0.15722656, ..., -0.29101562,
        -0.19921875,  0.265625  ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [38]:
y = df.OpenStatus.values

In [43]:
type(X)

numpy.ndarray

In [41]:
X.shape

(10000, 200, 300)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
X_test.shape

(2000, 200, 300)

In [51]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [53]:
# Define LSTM model architecture
lstm_model = Sequential()
lstm_model.add(LSTM(64, input_shape=(X.shape[1], X.shape[2]), activation='relu'))
lstm_model.add(Dense(5, activation='softmax'))

In [54]:
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
250/250 [==============================] - 15s 57ms/step - loss: nan - accuracy: 0.5052 - val_loss: nan - val_accuracy: 0.4885
Epoch 2/10
250/250 [==============================] - 13s 53ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 3/10
250/250 [==============================] - 13s 52ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 4/10
250/250 [==============================] - 13s 52ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 5/10
250/250 [==============================] - 13s 52ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 6/10
250/250 [==============================] - 13s 52ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 7/10
250/250 [==============================] - 13s 52ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 8/10
250/250 [==============

In [55]:
print(lstm_model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 64)                93440     
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 93,765
Trainable params: 93,765
Non-trainable params: 0
_________________________________________________________________
None


In [57]:
from tensorflow.keras.initializers import glorot_uniform
lstm_model = Sequential()
lstm_model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), activation='relu', kernel_initializer=glorot_uniform(), bias_initializer='zeros'))
lstm_model.add(Dense(5, activation='sigmoid'))


lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

lstm_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)


print(lstm_model.summary())

Epoch 1/10
250/250 [==============================] - 26s 101ms/step - loss: nan - accuracy: 0.5036 - val_loss: nan - val_accuracy: 0.4885
Epoch 2/10
250/250 [==============================] - 24s 97ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 3/10
250/250 [==============================] - 26s 105ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 4/10
250/250 [==============================] - 26s 103ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 5/10
250/250 [==============================] - 25s 102ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 6/10
250/250 [==============================] - 25s 99ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 7/10
250/250 [==============================] - 25s 100ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 8/10
250/250 [=========

In [59]:
#from scratch
X = df.drop('OpenStatus',axis=1)
y=df['OpenStatus']

In [60]:
print(X.shape)
print(y.shape)

(10000, 1)
(10000,)


In [72]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Bidirectional

vocab_size=10000

messages=X.copy()


In [74]:
messages.reset_index(inplace=True)

In [75]:
df

,OpenStatus,Combined_input
23330,0,android widget bitmap size hallo write widget ...
45699,2,stackoverflow error find the stackoverflow err...
128700,2,exact cost the instanc for hour amazon web ser...
118080,0,jqueri focu input got one main search field al...
74160,0,get the sub index foreach loop simplexml xml v...
...,...,...
23908,0,differ return valu from call store proc from d...
84748,1,record migrat error sap bydesign tri migrat su...
11692,3,spoon studio crash plea suggest anoth all one ...
116280,0,bing map silverlight bind for move target want...


In [76]:
messages['Combined_input'][1]

'stackoverflow error find the stackoverflow error unexpectedli defin putchar defin print main print exit defin actual the can ani other number that the result still the not refer the defin macro whi admin make what your code make more cool pure code http statu code'

In [82]:
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]',' ',messages['Combined_input'][i])
    review = review.lower()
    review = review.split()
    
    review = [stemmer.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075


3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725


5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368


6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7019
7020
7021
7022
7023


8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617
8618
8619
8620
8621
8622
8623
8624
8625
8626
8627
8628
8629
8630
8631
8632
8633
8634
8635
8636
8637
8638
8639
8640
8641
8642
8643
8644
8645
8646
8647
8648
8649
8650
8651
8652
8653
8654
8655
8656
8657
8658
8659
8660
8661
8662


In [83]:
corpus

['android widget bitmap size hallo write widget unsur size bitmap use background screen resolut densiti accord android develop app widget tutori find minimum width height densiti independ pixel use thi formula number cell follow thi formula use height one cell width four cell widget size bitmap need ldpi mdpi hdpi drawabl also need ani particular support screen set manifest tia frink android bitmap android widget android layout android manifest',
 'stackoverflow error find stackoverflow error unexpectedli defin putchar defin print main print exit defin actual ani number result still refer defin macro whi admin make code make cool pure code http statu code',
 'exact cost instanc hour amazon web servic foe window linux want know exact cost window linux instanc amazon price list show per hour window per hour linux price calcul show per hour month price calcul actual per hour one correct beacau lot money creat instanc test today want ani link ani friend tel exact cost alon hour instanc win

In [84]:
one_hot_repr=[one_hot(words,vocab_size)for words in corpus]
one_hot_repr

[[6381,
  2572,
  8464,
  3521,
  5953,
  3563,
  2572,
  9643,
  3521,
  8464,
  3580,
  5380,
  2921,
  5795,
  6315,
  3386,
  6381,
  1040,
  569,
  2572,
  7880,
  76,
  7739,
  4264,
  2694,
  6315,
  7270,
  8106,
  3580,
  2869,
  2079,
  3977,
  4461,
  361,
  2869,
  2079,
  3580,
  2694,
  2917,
  4461,
  4264,
  9136,
  4461,
  2572,
  3521,
  8464,
  8905,
  408,
  2449,
  4520,
  3572,
  6934,
  8905,
  3446,
  7842,
  5738,
  2921,
  4099,
  3595,
  6234,
  9266,
  6381,
  8464,
  6381,
  2572,
  6381,
  9852,
  6381,
  3595],
 [9004,
  2078,
  76,
  9004,
  2078,
  9032,
  7571,
  9925,
  7571,
  2769,
  9498,
  2769,
  4667,
  7571,
  1980,
  3446,
  3977,
  3547,
  178,
  6824,
  7571,
  9828,
  990,
  9930,
  6659,
  8708,
  6659,
  5967,
  8985,
  8708,
  3955,
  769,
  8708],
 [2606,
  9747,
  8233,
  4461,
  561,
  4270,
  9385,
  1767,
  2998,
  3384,
  4086,
  1935,
  2606,
  9747,
  2998,
  3384,
  8233,
  561,
  9874,
  7166,
  8771,
  4270,
  4461,
  2998,
  

In [136]:
from tensorflow.keras.utils import pad_sequences

In [169]:
sentence_length=40
embedded_docs=pad_sequences(one_hot_repr,padding='post',maxlen=sentence_length)
print(embedded_docs)

[[2869 2079 3977 ... 9852 6381 3595]
 [9004 2078   76 ...    0    0    0]
 [4270 4461 2998 ... 2998 3384 7965]
 ...
 [2171 4857 2917 ... 4079 8115 2996]
 [8850 7724   49 ... 4949 5414 9483]
 [1521 7335 5001 ... 9314 2869 1311]]


In [138]:
embedded_docs[0]

array([2869, 2079, 3977, 4461,  361, 2869, 2079, 3580, 2694, 2917, 4461,
       4264, 9136, 4461, 2572, 3521, 8464, 8905,  408, 2449, 4520, 3572,
       6934, 8905, 3446, 7842, 5738, 2921, 4099, 3595, 6234, 9266, 6381,
       8464, 6381, 2572, 6381, 9852, 6381, 3595], dtype=int32)

In [139]:
embedding_vector_features=50
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sentence_length))
model.add(LSTM(100))
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [140]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 40, 50)            500000    
                                                                 
 lstm_10 (LSTM)              (None, 100)               60400     
                                                                 
 dense_8 (Dense)             (None, 5)                 505       
                                                                 
Total params: 560,905
Trainable params: 560,905
Non-trainable params: 0
_________________________________________________________________


In [141]:
embedding_vector_features=50
model1=Sequential()
model1.add(Embedding(vocab_size,embedding_vector_features,input_length=sentence_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(5,activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [142]:
model1.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 40, 50)            500000    
                                                                 
 bidirectional_2 (Bidirectio  (None, 200)              120800    
 nal)                                                            
                                                                 
 dense_9 (Dense)             (None, 5)                 1005      
                                                                 
Total params: 621,805
Trainable params: 621,805
Non-trainable params: 0
_________________________________________________________________


In [143]:
len(embedded_docs),y.shape

(10000, (10000,))

In [144]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [145]:
X_final.shape,y_final.shape

((10000, 40), (10000,))

In [146]:
y_final.shape

(10000,)

In [147]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

In [148]:
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

In [149]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)
model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
125/125 [==============================] - 4s 26ms/step - loss: 1.2778 - accuracy: 0.5213 - val_loss: 1.1970 - val_accuracy: 0.5390
Epoch 2/10
125/125 [==============================] - 3s 23ms/step - loss: 1.0802 - accuracy: 0.5792 - val_loss: 1.1811 - val_accuracy: 0.5475
Epoch 3/10
125/125 [==============================] - 3s 24ms/step - loss: 0.9645 - accuracy: 0.6223 - val_loss: 1.1746 - val_accuracy: 0.5445
Epoch 4/10
125/125 [==============================] - 3s 23ms/step - loss: 0.8494 - accuracy: 0.6743 - val_loss: 1.2957 - val_accuracy: 0.5425
Epoch 5/10
125/125 [==============================] - 3s 23ms/step - loss: 0.7362 - accuracy: 0.7330 - val_loss: 1.3797 - val_accuracy: 0.5290
Epoch 6/10
125/125 [==============================] - 3s 23ms/step - loss: 0.6159 - accuracy: 0.7836 - val_loss: 1.4215 - val_accuracy: 0.5055
Epoch 7/10
125/125 [==============================] - 3s 24ms/step - loss: 0.5090 - accuracy: 0.8257 - val_loss: 1.6173 - val_accuracy: 0.5145

In [150]:
embedding_vector_features=50
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sentence_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [151]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 40, 50)            500000    
                                                                 
 dropout (Dropout)           (None, 40, 50)            0         
                                                                 
 lstm_12 (LSTM)              (None, 100)               60400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_10 (Dense)            (None, 5)                 505       
                                                                 
Total params: 560,905
Trainable params: 560,905
Non-trainable params: 0
_________________________________________________________________


In [152]:
embedding_vector_features=50
model1=Sequential()
model1.add(Embedding(vocab_size,embedding_vector_features,input_length=sentence_length))
model.add(Dropout(0.2))
model1.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.2))
model1.add(Dense(5,activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [153]:
model1.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 40, 50)            500000    
                                                                 
 bidirectional_3 (Bidirectio  (None, 200)              120800    
 nal)                                                            
                                                                 
 dense_11 (Dense)            (None, 5)                 1005      
                                                                 
Total params: 621,805
Trainable params: 621,805
Non-trainable params: 0
_________________________________________________________________


In [168]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=64)
model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=64)

Epoch 1/15
125/125 [==============================] - 3s 24ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 2/15
125/125 [==============================] - 3s 26ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 3/15
125/125 [==============================] - 3s 25ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 4/15
125/125 [==============================] - 3s 24ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 5/15
125/125 [==============================] - 3s 25ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 6/15
125/125 [==============================] - 3s 26ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 7/15
125/125 [==============================] - 3s 24ms/step - loss: nan - accuracy: 0.5058 - val_loss: nan - val_accuracy: 0.4885
Epoch 8/15
125/125 [=====================

In [170]:
y_pred=model.predict(X_test)
y_pred1=model1.predict(X_test)

63/63 [==============================] - 0s 7ms/step


In [171]:
print(type(y_test))
print(type(y_pred1))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [172]:
from sklearn.metrics import classification_report
y_pred1_int = np.round(y_pred1)
print(classification_report(y_test, y_pred1_int))

              precision    recall  f1-score   support

           0       0.59      0.65      0.62       977
           1       0.34      0.31      0.32       471
           2       0.35      0.28      0.31       261
           3       0.38      0.27      0.32       217
           4       0.06      0.04      0.05        74

   micro avg       0.48      0.46      0.47      2000
   macro avg       0.35      0.31      0.32      2000
weighted avg       0.46      0.46      0.45      2000
 samples avg       0.46      0.46      0.46      2000



/Users/vipin-16319/opt/anaconda3/envs/MLEnvi/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
